### Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.datasets import ImageFolder
from torchvision.models import resnet101, ResNet101_Weights
import torch.optim as optim
import torch.nn as nn
from PIL import Image
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2

### Data Preprocessing

In [ ]:
# transformation 정의
transform = transforms.Compose([
    transforms.Resize(size=224),
    transforms.RandomHorizontalFlip(p=0.5), # 50% 확률로 horizontal flip
    transforms.RandomRotation(degrees=15), # -15 ~ 15도 랜덤하게 회전
    transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.5, 1.0)), # 가우시안 블러 추가
    transforms.CenterCrop(size=175), # 중앙 부분 자르기
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # 색상 정규화
])

# 데이터셋 전처리
data_path = '/content/drive/MyDrive/UTKFace_data'
dataset = ImageFolder(root=data_path, transform=transform) # ImageFolder는 subfolder의 이름을 클래스로 사용

# 훈련/검증/테스트 세트 나누기
train_size = int(0.65 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

### Load pre-trained ResNet50 model



In [ ]:
num_classes = 5 # [Asian, Black, Indian, Others, White]
#model = resnet101(weights=ResNet101_Weights.DEFAULT) # ImageNet 데이터로 사전 훈련된 모델 가져오기
#model.fc = nn.Linear(model.fc.in_features, num_classes) # fine-tuning을 위해 마지막 층의 아웃풋은 인종 수로 설정

# 기존 모델을 불러올 거면 아래 코드 사용
model = torch.load('/content/drive/MyDrive/ResNet/resnet_checkpoints_v3/transformation2/model_epoch_2.pth')

# 과적합 방지를 위해 기존 레이어 freeze
for param in model.parameters():
    param.requires_grad = False

# 레이어 unfreeze -> 맨 마지막 레이어부터 하나씩 unfreeze 하면서 실험하고 가장 좋은 결과로 써야 할 듯!
layers_unfrozen = 3 # 0 ~ 4로 테스팅 -> 3이 베스트
blocks = [model.layer2, model.layer3, model.layer4]
for block in blocks:
    for param in block.parameters():
        param.requires_grad = True

model.fc.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001) # Adam으로 먼저 해보고 괜찮으면 킵. 아니면 SGD -> 실험 결과 Adam으로 했을 때 acc 0.15%p 더 높음

print(model)

<ipython-input-7-0f4fbba17337>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/content/drive/MyDrive/ResNet/resnet_checkpoints_v3/transformation2/model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
latest_checkpoint = 1
num_epochs = 50

device = torch.device('cuda' if torch.cuda.is_available() else torch.device("cpu"))
model.to(device)

for epoch in range(latest_checkpoint, num_epochs):
    model.train()
    running_loss = 0.0

    # 학습 단계
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    model_save_path = f"/content/drive/MyDrive/ResNet/resnet_checkpoints_v3/transformation2/model_epoch_{epoch+1}.pth"
    torch.save(model, model_save_path)
    print(f"Model saved to {model_save_path}")

    # 검증 단계
    model.eval()
    val_loss = 0.0
    corrects = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels.data)

    val_loss /= len(val_loader.dataset)
    val_acc = corrects.double() / len(val_loader.dataset)
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")

### Find percentage

In [ ]:
!pip install facenet-pytorch # 약 1분 소요

In [ ]:
from facenet_pytorch import MTCNN

# 모델 불러오기
model = torch.load('/content/drive/MyDrive/ResNet/resnet_checkpoints_v2/unfrozen=3/model_epoch_40.pth', map_location=torch.device('cpu'))
model.eval()

# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 예측 함수
def predict_ethnicity(image):
    image = transform(image)
    image = image.unsqueeze(0)

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    return predicted.item()

# MTCNN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)

# 이미지 로딩
image_path = '/content/drive/MyDrive/test/01.jpg'
img_array = cv2.imread(image_path)
img = Image.fromarray(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))

# 얼굴 검출
boxes, probs = mtcnn.detect(img)

if boxes is not None:
    asians = 0
    for box in boxes:
        # 얼굴 크롭
        face_region = img.crop((box[0], box[1], box[2], box[3]))
        if predict_ethnicity(face_region) == 0:
            asians += 1
            flag = True
        else:
            flag = False

        # 바운딩 박스 그리기
        if flag: color = (0, 0, 255)
        else: color = (0, 255, 0)
        cv2.rectangle(img_array, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), color, 2)

    cv2_imshow(img_array)
    print(f"Asians: {asians} ({asians / len(boxes) * 100:.2f}%)")